In [2]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
#from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config
import time
from ctapipe.image import number_of_islands





In [2]:
int_cut = 0.3
min_pixel = 70

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

all_events = 0
stereo = 0
event_container_protons = {}
t1 = time.perf_counter(), time.process_time()
check_events = ['78000', '78001', '78002']
for k in range(86,100):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run39{k}.simtel.gz'
    print(filename)
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,event in enumerate(event_iterator):
        #print(str(int(k*1000+i)))
        #print(str(int(k*1000+i)) in check_events)
        all_events += 1
        if len(event.trigger.tels_with_trigger) > 1:  
            stereo += 1          
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
            #muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            calib(event)
            image_processor(event)
            muon_processor(event)
            for m in event.muon.tel.keys():
                if not np.isnan(event.muon.tel[m].efficiency.width):
                    event_container_protons[keks+i] = event
    #t2 = time.perf_counter(), time.process_time()
        

#print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
#print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run3986.simtel.gz


TelescopeParameter type argument 'MST_*' did not match any known telescope types
TelescopeParameter type argument 'SST_1M_*' did not match any known telescope types
TelescopeParameter type argument 'MST_*' did not match any known telescope types
TelescopeParameter type argument 'SST_1M_*' did not match any known telescope types
TelescopeParameter type argument 'MST_*' did not match any known telescope types
TelescopeParameter type argument 'SST_1M_*' did not match any known telescope types
TelescopeParameter type argument 'MST_*' did not match any known telescope types
TelescopeParameter type argument 'SST_1M_*' did not match any known telescope types
TelescopeParameter type argument 'MST_*' did not match any known telescope types
TelescopeParameter type argument 'SST_1M_*' did not match any known telescope types
TelescopeParameter type argument 'MST_*' did not match any known telescope types
TelescopeParameter type argument 'SST_1M_*' did not match any known telescope types
TelescopeP

KeyboardInterrupt: 

In [19]:
event.simulation.shower.energy.to_value()

0.07547195255756378

In [32]:
if event.r1.tel[1].waveform is not None:
    print(True)

True


In [12]:
print("all events = ", all_events)
print("stereo = ", stereo)
print("muons = ", len(event_container_protons))
print("muons events = ", event_container_protons.keys())
#for i in event_container_protons:
#    print(i)

opt_eff_p = []
for i in event_container_protons:
    for m in event_container_protons[i].trigger.tels_with_trigger:
        if not np.isnan(event_container_protons[i].muon.tel[m].efficiency.optical_efficiency) and event_container_protons[i].muon.tel[m].efficiency.optical_efficiency < 100 and event_container_protons[i].muon.tel[m].parameters.completeness > 0.5 and event_container_protons[i].muon.tel[m].parameters.mean_squared_error.to_value() < 0.01 and event_container_protons[i].dl1.tel[m].parameters.morphology.n_pixels > 50:
            opt_eff_p.append(event_container_protons[i].muon.tel[m].efficiency.optical_efficiency)

print("mean = ", np.mean(opt_eff_p))
print("std = ", np.std(opt_eff_p))
print("len = ", len(opt_eff_p))


all events =  10363
stereo =  4197
muons =  25
muons events =  dict_keys([86287, 86345, 86633, 87027, 87511, 87597, 88049, 88103, 88130, 88524, 88598, 88599, 88647, 89104, 89236, 90324, 90493, 90635, 91012, 91662, 94747, 96593, 98657, 99076, 99694])
mean =  0.21781278887705777
std =  0.03717869813634657
len =  22


In [ ]:
#
#
# Images for events without cleaning
#
#

x_range_1 = [i for i in range(len(opt_eff_p))]
#std_arr = [np.std(opt_eff_p)/np.mean(opt_eff_p) for i,j in enumerate(opt_eff_p)]
std_arr_1 = [np.std(opt_eff_p)/i**0.5 for i,j in enumerate(opt_eff_p)]
std_arr_1_big = [np.std(opt_eff_p) for i,j in enumerate(opt_eff_p)]
#std_arr = [np.std(opt_eff) for i,j in enumerate(opt_eff)]


mean_range_1 = [i-1 for i in range(len(opt_eff_p)+3)]
y_upper_1 = [np.mean(opt_eff_p)+np.std(opt_eff_p) for i in mean_range_1]
y_lower_1 = [np.mean(opt_eff_p)-np.std(opt_eff_p) for i in mean_range_1]

y_mid_1 = [np.mean(opt_eff_p) for i in mean_range_1]
#y_mid_1 = [np.mean(opt_eff_p[:i+1]) for i in range(len(opt_eff_p[:i+1]))]


#y_upper_1 = [j+std_arr_1[-1] for i,j in enumerate(y_mid_1)]
#y_lower_1 = [j-std_arr_1[-1] for i,j in enumerate(y_mid_1)]

#y_upper_1 = [j+std_arr_1[-1] for i,j in enumerate(y_mid_1)]
#y_lower_1 = [j-std_arr_1[-1] for i,j in enumerate(y_mid_1)]


#y_upper_1 = [j+np.std(opt_eff_p[:i+1]) for i,j in enumerate(y_mid_1)]
#y_lower_1 = [j-np.std(opt_eff_p[:i+1]) for i,j in enumerate(y_mid_1)]


y_upper_1 = [j+std_arr_1[-1] for i,j in enumerate([np.mean(opt_eff_p) for i in x_range_1])]
y_lower_1 = [j-std_arr_1[-1] for i,j in enumerate([np.mean(opt_eff_p) for i in x_range_1])]

plt.figure(figsize = (12,6))
plt.ylim(0,0.4)

plt.style.use('default')
plt.xlim(-1,len(opt_eff_p))
plt.xlabel("Event")
#plt.ylabel("Optical eff with rel. error for 1 measurment")
plt.ylabel("Optical efficiency with std")
#plt.title("Opt eff for muon rings without performing additional cleaning int_cut<0.1")
plt.title("Events before additional cleaning (pix > 50, intensity_cut = 0.3, completeness>0.5)")
plt.grid(alpha=0.5)
#plt.ylim(0,0.4)

plt.scatter(x_range_1, opt_eff_p)
plt.errorbar(x_range_1, opt_eff_p, yerr=std_arr_1, fmt='o', ms = 7, alpha = 0.8, color = 'darkorange')#, c = 'blue')#, s = 100)#, c= 'black')


#plt.fill_between(x_range_1, y_upper_1, y_lower_1, color = 'green', alpha = 0.3, label = 'mean +/- $1\sigma$')
plt.fill_between(x_range_1, y_upper_1, y_lower_1, color = 'green', alpha = 0.3, label = 'mean +/- $1\sigma$')
#plt.plot(x_range_1, y_mid_1, label = "mean", c = 'green', linewidth = 1.5)
plt.plot(x_range_1, [np.mean(opt_eff_p) for i in x_range_1], label = "mean", c = 'green', linewidth = 1.5)


plt.legend()
print("mean = ", np.mean(opt_eff_p))
print("std = ", np.std(opt_eff_p))
print("rel_err = ", np.std(opt_eff_p)/np.mean(opt_eff_p))
print("len = ", len(opt_eff_p))
#plt.savefig("/Users/vdk/ImagePurgatorium/4LSTprotonsWithoutCleaningIntCut0.3_Containment>0.5_RelStd_ArtificialGood.png")

In [ ]:
#
#
# Need this to fill optical efficiency array with new values
#
#
optical_coef = []
event_optical_coef = []
for i in event_container_protons:
    for m in event_container_protons[i].trigger.tel.keys():
        #print(np.isnan(event_container[i].muon.tel[m].efficiency.optical_efficiency))
        #if cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.3 and cleaned_event_container[i].muon.tel[m].parameters.completeness > 0.5:
        if not np.isnan(event_container_protons[i].muon.tel[m].efficiency.optical_efficiency) and event_container_protons[i].dl1.tel[m].parameters.morphology.n_pixels > 50:
            optical_coef.append(event_container_protons[i].muon.tel[m].efficiency.optical_efficiency)
            event_optical_coef.append(i)

print(np.mean(optical_coef))
print(np.std(optical_coef))
print(np.std(optical_coef)/np.mean(optical_coef))
print(len(optical_coef))
#optical_coef = optical_coef[0:31]


#
#
# Code for extracting events which are presented in both cleaned and raw dataset
#
#

# dictionary with event:opticalefficiency for original events
res_dict = {event_optical_coef[i]:optical_coef[i] for i in range(len(optical_coef))}
# dictionary with event:opticalefficiency for cleaned events
res_dict_cleaned = {events_nice_opt_eff[i]:nice_opt_eff[i] for i in range(len(nice_opt_eff))}
print(len(res_dict))
print(len(res_dict_cleaned))

plot = []
for i in res_dict_cleaned.keys():
    if i in res_dict.keys():
        plot.append(res_dict[i])
    else:
        plot.append(0)

new_plot = []
new_ind = []
new_std = []
for i,j in enumerate(plot):
    if not j == 0:
        new_plot.append(j)
        new_ind.append(i)
        new_std.append(np.std(new_plot))
        


        
print("len of remained = ", len(new_plot))
print("len of all = ", len(nice_opt_eff))

cumulative_dif = 0

for i in res_dict:
    if i in res_dict_cleaned:
        cumulative_dif += abs(res_dict[i] - res_dict_cleaned[i])

print("cumulative_dif = ", cumulative_dif)

print("relative cumulative_dif = ", cumulative_dif/len(new_plot))

In [ ]:
#
#
# Images for events after cleaning
#
#

#plt.style.use('tableau-colorblind10')
plt.style.use('default')


#std_arr_3 = []



mean_range = [i-1 for i in range(len(nice_opt_eff)+2)]
mean_range = [i-1 for i in range(len(nice_opt_eff)+2)]

y_mid = [np.mean(nice_opt_eff) for i in mean_range]
#y_mid = [np.mean(nice_opt_eff[:i+1]) for i in range(len(nice_opt_eff[:i+1]))]


y_upper = [np.mean(nice_opt_eff)+np.std(nice_opt_eff) for i in mean_range]
y_lower = [np.mean(nice_opt_eff)-np.std(nice_opt_eff) for i in mean_range]



x_range = [i for i in range(len(nice_opt_eff))]




#std_arr = [np.std(optical_coef)/np.mean(optical_coef) for i,j in enumerate(optical_coef)]
std_arr = [np.std(nice_opt_eff)/i**0.5 for i,j in enumerate(nice_opt_eff)]
std_arr_big = [np.std(nice_opt_eff) for i,j in enumerate(nice_opt_eff)]

std_arr_3 = [np.std(new_plot)/i**0.5 for i,j in enumerate(new_plot)]
std_arr_3_big = [np.std(new_plot) for i,j in enumerate(new_plot)]
#std_arr = [np.std(opt_eff) for i,j in enumerate(opt_eff)]


plt.figure(figsize = (12,6))
plt.ylim(0.0,0.4)
#plt.ylim(0.095,0.265)
plt.xlabel("Event")
#plt.ylabel("Optical eff with rel. error for 1 measurment")
plt.ylabel("Optical efficiency value")
#plt.title("Opt eff for muon rings after time sample cleaning, pix > 50, intensity<0.2")
plt.title("Events after additional cleaning (pix > 50, intensity_cut = 0.3, time_bin<17, completeness>0.5)")
#plt.title("Events after island cleaning (size $\leq 4$)")


plt.grid(alpha=0.5)
#plt.ylim(0.05,0.3)
#plt.scatter(new_ind, new_plot, color = 'orange')


#y_upper = [j+std_arr[-1] for i,j in enumerate(y_mid)]
#y_lower = [j-std_arr[-1] for i,j in enumerate(y_mid)]


y_upper = [j+std_arr[-1] for i,j in enumerate([np.mean(nice_opt_eff) for i in x_range])]
y_lower = [j-std_arr[-1] for i,j in enumerate([np.mean(nice_opt_eff) for i in x_range])]

plt.errorbar(x_range, nice_opt_eff, yerr=std_arr, fmt='o', ms = 5, alpha = 0.8, label="cleaned")#, c = 'blue')#, s = 100)#, c= 'black')
#plt.errorbar(new_ind, new_plot, yerr=std_arr_3, fmt='o', ms = 5, alpha = 0.8, label="original") #, c = 'blue')#, s = 100)#, c= 'black')
#plt.errorbar(new_ind, new_plot, yerr=new_std, fmt='o', ms = 5, alpha = 0.8, label="original") #, c = 'blue')#, s = 100)#, c= 'black')

#plt.scatter(new_ind, new_plot, alpha = 0.9, color = "orange", label="original") #, c = 'blue')#, s = 100)#, c= 'black')


#plt.fill_between(mean_range, y_upper, y_lower, alpha = 0.3, color = 'green', label = 'mean +/- $1\sigma$')
plt.fill_between(x_range, y_upper, y_lower, alpha = 0.3, color = 'green', label = 'mean +/- $1\sigma$')
#plt.plot(mean_range, y_mid, label = "mean", lw = 1.5, color='green')

#plt.plot(x_range, y_mid, label = "mean", lw = 1.5, color='green')
plt.plot(x_range, [np.mean(nice_opt_eff) for i in x_range], label = "mean", lw = 1.5, color='green')

#plt.fill_between(x_range_1, y_upper_1, y_lower_1, color = 'r', alpha = 0.3, label = 'mean +/- $1\sigma$')

plt.xlim(-1,len(nice_opt_eff))
plt.legend()
print("mean = ", np.mean(nice_opt_eff))
print("std = ", np.std(nice_opt_eff))
print("rel_err = ", np.std(nice_opt_eff)/np.mean(nice_opt_eff))
print("len of all = ", len(nice_opt_eff))
print("len of remained = ", len(new_plot))
#plt.savefig("/Users/vdk/ImagePurgatorium/CleanedOptEff_pix>50_int=01_16timebin_complet>05_RelStdErr_fdsfs.png")
#plt.savefig("/Users/vdk/ImagePurgatorium/IslandsCleaned_maskMEAN_intcut30_v2.png", dpi=900)

### Cleaning with flagging

In [ ]:
# TIME CLEANING ALGO + ISLANDS CLEANING

int_cut = 0.3
min_pixel = 160

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels < {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
cleaned_event_container = {}
r1_cut = 2
min_pixel_cut = 60
max_pixel_cut = 200
low_threshold_variance = 0.1
up_threshold_variance = 0.4
max_time = 14

for k in range(78,100):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run39{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    keks = 1000*k
    for i,event in enumerate(event_iterator):

        if len(event.trigger.tels_with_trigger) > 1:
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config_2)
            #muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            for tels in event.trigger.tels_with_trigger:

                tmp_arr = []
                var_arr = []
                for pix_id in event.r1.tel[tels].waveform[:,10:max_time]:
                    tmp_arr.append(np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
                    var_arr.append(np.var(pix_id))
            
                tmp_arr = np.array(tmp_arr)
                pixel_number = len(tmp_arr[tmp_arr>r1_cut])
            
            if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut) and (np.mean(var_arr) >low_threshold_variance) and (np.mean(var_arr) < up_threshold_variance):
                for tels in event.trigger.tels_with_trigger:

                    pixel_mask = []
                    original_r1_waveform = event.r1.tel[tels].waveform.copy()
                    for_clean_mask = []

                    for k,l in enumerate(event.r0.tel[tels].waveform[1]):
                        if np.argmax(l) in range(17,41): 
                            pixel_mask.append(True)
                        else:
                            pixel_mask.append(False)

                    working_wave = event.r1.tel[tels].waveform.copy()
                    for k,l in enumerate(working_wave):
                        if pixel_mask[k]:
                            for_clean_mask.append(True)
                            #working_wave[k] = np.zeros(40)     
                            working_wave[k] = np.mean(event.r1.tel[tels].waveform, axis = 0)
                        else:
                            for_clean_mask.append(False)
                    
                    event.r1.tel[tels].waveform = working_wave

                    calib(event)
                    image_processor(event)
                    cam_geometry = source.subarray.tel[tels].camera.geometry
                    return_pixels = np.array([])
                    for_clean_mask = np.array(for_clean_mask)
                    n_islands, island_id = number_of_islands(cam_geometry, for_clean_mask)

                    for t in range(1,n_islands+1):
                        if len(np.where(island_id == t)[0]) <= 3:
                            return_pixels = np.append(return_pixels,np.where(island_id ==t)[0])

                    return_pixels = return_pixels.astype(int)

                    event.r1.tel[tels].waveform[return_pixels,:] = original_r1_waveform[return_pixels,:]  

                    calib(event)
                    image_processor(event)
                    muon_processor(event)   
                        
                    if len(event.muon.tel.keys()):       
                        for m in event.muon.tel.keys():
                            if not np.isnan(event.muon.tel[m].efficiency.width):
                                cleaned_event_container[keks+i] = event   






### Without flagging

In [16]:
# TIME CLEANING ALGO + ISLANDS CLEANING WITHOUT FLAGGING

int_cut = 0.3
min_pixel = 20

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
cleaned_event_container = {}
r1_cut = 2
min_pixel_cut = 60
max_pixel_cut = 200
low_threshold_variance = 0.1
up_threshold_variance = 0.4
max_time = 14

for k in range(86,100):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run39{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    keks = 1000*k
    for i,event in enumerate(event_iterator):

        if len(event.trigger.tels_with_trigger) > 1:
            image_processor = ImageProcessor(source.subarray)
            #muon_processor = MuonProcessor(source.subarray, config = muon_processor_config_2)
            muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            for tels in event.trigger.tels_with_trigger:
                pixel_mask = []
                for_clean_mask = []
                
                original_r1_waveform = event.r1.tel[tels].waveform.copy()
                
                for k,l in enumerate(event.r1.tel[tels].waveform):
                    if np.argmax(l) in range(16,41): 
                        pixel_mask.append(True)
                    else:
                        pixel_mask.append(False)

                working_wave = event.r1.tel[tels].waveform.copy()
                for k,l in enumerate(working_wave):
                    if pixel_mask[k]:
                        for_clean_mask.append(True)
                        #working_wave[k] = np.zeros(40)     
                        working_wave[k] = np.mean(event.r1.tel[tels].waveform, axis = 0)
                    else:
                        for_clean_mask.append(False)
                event.r1.tel[tels].waveform = working_wave

                calib(event)
                image_processor(event)
                cam_geometry = source.subarray.tel[tels].camera.geometry
                return_pixels = np.array([])
                for_clean_mask = np.array(for_clean_mask)
                n_islands, island_id = number_of_islands(cam_geometry, for_clean_mask)

                for t in range(1,n_islands+1):
                    if len(np.where(island_id == t)[0]) <= 3:
                        return_pixels = np.append(return_pixels,np.where(island_id ==t)[0])

                return_pixels = return_pixels.astype(int)

                event.r1.tel[tels].waveform[return_pixels,:] = original_r1_waveform[return_pixels,:]  

                calib(event)
                image_processor(event)
                muon_processor(event)   
                    
                if len(event.muon.tel.keys()):       
                    for m in event.muon.tel.keys():
                        if not np.isnan(event.muon.tel[m].efficiency.width):
                            cleaned_event_container[keks+i] = event   






Falling back to gzip module
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
Falling back to gzip module
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/

E VariableMetricBuilder Initial matrix not pos.def.


Falling back to gzip module
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
Falling back to gzip module
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/

E VariableMetricBuilder Initial matrix not pos.def.


/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta


E VariableMetricBuilder Initial matrix not pos.def.
E VariableMetricBuilder Initial matrix not pos.def.
E VariableMetricBuilder Initial matrix not pos.def.
E VariableMetricBuilder Initial matrix not pos.def.


/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
Falling back to gzip module
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta


E VariableMetricBuilder Initial matrix not pos.def.
E VariableMetricBuilder Initial matrix not pos.def.


Falling back to gzip module
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likelihood.py:98: RuntimeWarning: invalid value encountered in log
  neg_log_l = np.log(theta + EPSILON) + (image - prediction) ** 2 / theta
/Users/vdk/mambaforge/envs/cta/lib/python3.11/site-packages/ctapipe/image/pixel_likeliho

E VariableMetricBuilder Initial matrix not pos.def.


In [ ]:
len(cleaned_event_container)

In [15]:
nice_events = []
nice_opt_eff = []
events_nice_opt_eff = []

for i in cleaned_event_container:
    for m in cleaned_event_container[i].trigger.tel.keys():
        if not np.isnan(cleaned_event_container[i].muon.tel[m].efficiency.width):
            if cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.3 and cleaned_event_container[i].muon.tel[m].parameters.completeness > 0.5 and cleaned_event_container[i].muon.tel[m].parameters.mean_squared_error.to_value() < 0.01:
                nice_events.append(i)
                nice_opt_eff.append(cleaned_event_container[i].muon.tel[m].efficiency.optical_efficiency)
                events_nice_opt_eff.append(i)

print("mean ", np.mean(nice_opt_eff))
print("std ", np.std(nice_opt_eff))
print("len ", len(nice_opt_eff))

mean  0.32896087904880167
std  0.47222347570581547
len  21


In [ ]:
nice_events = []
nice_opt_eff = []
for i in cleaned_event_container:
    for m in cleaned_event_container[i].muon.tel.keys():
        if not np.isnan(cleaned_event_container[i].muon.tel[m].efficiency.width):
            if cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 0 and abs(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.3 and cleaned_event_container[i].muon.tel[m].parameters.completeness > 0.5 and cleaned_event_container[i].muon.tel[m].parameters.mean_squared_error.to_value() < 0.01:
                #print("i = ", i, "mse = ", cleaned_event_container[i].muon.tel[m].parameters.mean_squared_error.to_value())
                nice_events.append(i)
                nice_opt_eff.append(cleaned_event_container[i].muon.tel[m].efficiency.optical_efficiency)

print("mean ", np.mean(nice_opt_eff))
print("std ", np.std(nice_opt_eff))
print("len ", len(nice_opt_eff))         
check = 0
for i in nice_events:
    for tels in cleaned_event_container[i].muon.tel.keys():
        if not np.isnan(cleaned_event_container[i].muon.tel[tels].efficiency.optical_efficiency) and cleaned_event_container[i].muon.tel[tels].parameters.mean_squared_error.to_value() < 0.01:
            check += 1
            plt.figure(figsize = (9,6))
            camgeom = source.subarray.tel[tels].camera.geometry
            #print("i = ", i, "mse = ",cleaned_event_container[i].muon.tel[tels].parameters.mean_squared_error.to_value())
            #title=f"Event_{i}_optEff{round(event_container_protons[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container_protons[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container_protons[i].trigger.tel.keys())}"
            title=f"Cleaned_IslandRing_{i}_optEff{round(cleaned_event_container[i].muon.tel[tels].efficiency.optical_efficiency,3)}_width{round(cleaned_event_container[i].muon.tel[tels].efficiency.width.to_value(),3)}_triggeredTel{tels}"
            #title = f"number of pixel {cleaned_event_container[i].dl1.tel[tels].parameters.morphology.n_pixels}"
            disp = CameraDisplay(camgeom,title=title)
            disp.image = cleaned_event_container[i].simulation.tel[tels].true_image
            #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
            disp.cmap = plt.cm.RdBu_r
            disp.add_colorbar()
            #disp.set_limits_percent(95)
            plt.show()
            #plt.savefig(f"/Users/vdk/3978-4000runs_MuonsCleaning+Island17bin_intensitycut<0.3_event{i}.png")
    

In [ ]:
print(len(nice_events))
check

In [ ]:
energies_all = []
for k in range(90,100):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run39{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        if len(j.trigger.tels_with_trigger) > 1:
            energies_all.append(j.simulation.shower.energy)
energies_new = [i.to_value() for i in energies_all]
histplot(energies_new, log_scale = True, bins = 30)
plt.xlabel("Energy in TeV")
plt.title("Shower energies for 4 LST stereo proton trigger")
#plt.savefig("/Users/vdk/Shower_energy_4LST_20runs.png")
print(np.mean(energies_new))

In [ ]:
print(len(energies_new))
np.mean(energies_new)

In [ ]:
source.subarray.peek()

In [ ]:
# PROCESSING OF REGULAR IMAGES
lets_check = 0
for i in event_container_protons:
    for m in event_container_protons[i].trigger.tel.keys():
        if not np.isnan(event_container_protons[i].muon.tel[m].efficiency.width):
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                
                print("n_pixels ", event_container_protons[i].dl1.tel[m].parameters.morphology.n_pixels)
                #print("MSE ", event_container[i].muon.tel[m].parameters.mean_squared_error)                
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                #if event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.1:
                #if abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.2:
                #print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                lets_check+=1
                plt.figure(figsize = (9,6))
                camgeom = source.subarray.tel[m].camera.geometry
                title=f"Event_{i}_optEff{round(event_container_protons[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container_protons[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container_protons[i].trigger.tel.keys())}"
                #title=f"CleanedRing_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggeredTel{m}"
                disp = CameraDisplay(camgeom,title=title)
                disp.image = event_container_protons[i].simulation.tel[m].true_image
                #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
                disp.cmap = plt.cm.RdBu_r
                disp.add_colorbar()
                disp.set_limits_percent(95)
                #plt.show()
                #plt.savefig(f"/Users/vdk/3978-4000runs-LST4StereoMuons_intensitycut<0.3_event{i}.png")
                #nice_opt_eff.append(event_contaner[i].muon.tel[m].efficiency.optical_efficiency)
                    
#plt.close()

In [ ]:
opt_eff_p

In [ ]:
np.mean(opt_eff_p)

In [ ]:
# TIME CLEANING ALGO

int_cut = 0.3
min_pixel = 70

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
event_container_protons = {}
cleaned_event_container = {}
t1 = time.perf_counter(), time.process_time()

for k in range(90,100):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run39{k}.simtel.gz'
    print(filename)
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        checker += 1
        event = j
        if len(j.trigger.tels_with_trigger) > 1:            
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray)
            #muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            for tels in event.trigger.tels_with_trigger:
                pixel_mask = []
                for k,l in enumerate(event.r0.tel[tels].waveform[1]):
                    if np.argmax(l) in range(17,40): 
                        pixel_mask.append(True)
                    else:
                        pixel_mask.append(False)
                working_wave = event.r1.tel[tels].waveform.copy()
                for k,l in enumerate(working_wave):
                    if pixel_mask[k]:
                        #working_wave[k] = np.zeros(40)     
                        working_wave[k] = np.mean(event.r1.tel[tels].waveform, axis = 0)
                event.r1.tel[tels].waveform = working_wave
            
                calib(event)
                image_processor(event)
                muon_processor(event)
        for m in j.muon.tel.keys():
            if not np.isnan(j.muon.tel[m].efficiency.width):
                cleaned_event_container[keks+i] = j
    t2 = time.perf_counter(), time.process_time()
        

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

Process cleaned events and choose only muon rings based on several quality cuts

In [ ]:
nice_events = []
nice_opt_eff = []
for i in cleaned_event_container:
    for m in cleaned_event_container[i].trigger.tel.keys():
        if not np.isnan(cleaned_event_container[i].muon.tel[m].efficiency.width):
            if cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.3 and cleaned_event_container[i].muon.tel[m].parameters.completeness > 0.5 and cleaned_event_container[i].muon.tel[m].parameters.mean_squared_error.to_value() < 0.01:
                nice_events.append(i)
                nice_opt_eff.append(cleaned_event_container[i].muon.tel[m].efficiency.optical_efficiency)

In [ ]:
# 16 bin 
#
#For zero cleaning
# opt eff =  0.21178922297555983
# len =  6
#
# For mean cleaning
# opt eff =  0.2057269341304728
# len =  8
#
# + island
# opt eff  0.20622896281132624
# len  11
#
# 17 bin
# For zero
# opt eff =  0.20629052865678918
# len =  9
#
# + island
# opt eff  0.20260365420720844
# len  9
#
#
# For mean cleaning
# opt eff =  0.20096451720577296
# len =  9
#
# + island cleaner
# opt eff =  0.204597032282674
# len =  11
#
print("mean = ", np.mean(nice_opt_eff))
print("len = ", len(nice_opt_eff))

In [ ]:
print

In [ ]:
# TIME CLEANING ALGO + ISLANDS CLEANING

int_cut = 0.3
min_pixel = 150

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels < {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
cleaned_event_container = {}

for k in range(90,100):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run39{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,event in enumerate(event_iterator):
        
        image_processor = ImageProcessor(source.subarray)
        #muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)



        for tels in event.trigger.tels_with_trigger:
            pixel_mask = []
            original_r1_waveform = event.r1.tel[tels].waveform.copy()
            for_clean_mask = np.array([]).astype(bool)

            for k,l in enumerate(event.r0.tel[tels].waveform[1]):
                if np.argmax(l) in range(16,40): 
                    pixel_mask.append(True)
                else:
                    pixel_mask.append(False)

            working_wave = event.r1.tel[tels].waveform.copy()
            for k,l in enumerate(working_wave):
                if pixel_mask[k]:
                    for_clean_mask = np.append(for_clean_mask, True)
                    working_wave[k] = np.zeros(40)     
                    #working_wave[k] = np.mean(event.r1.tel[tels].waveform, axis = 0)
                else:
                    for_clean_mask = np.append(for_clean_mask, False)
            event.r1.tel[tels].waveform = working_wave

            calib(event)
            image_processor(event)
            cam_geometry = source.subarray.tel[tels].camera.geometry

        return_pixels = np.array([])

        n_islands, island_id = number_of_islands(cam_geometry, for_clean_mask)

        for t in range(1,n_islands+1):
            if len(np.where(island_id == t)[0]) <= 3:
                return_pixels = np.append(return_pixels,np.where(island_id ==t)[0])

        return_pixels = return_pixels.astype(int)

        event.r1.tel[tels].waveform[return_pixels,:] = original_r1_waveform[return_pixels,:]  

        calib(event)
        image_processor(event)
        muon_processor(event)   
                
        if len(event.muon.tel.keys()):       
            for m in event.muon.tel.keys():
                if not np.isnan(event.muon.tel[m].efficiency.width):
                    cleaned_event_container[keks+i] = event    



In [ ]:
len(cleaned_event_container)

In [ ]:
print("mean ", np.mean(nice_opt_eff))
print("std ", np.std(nice_opt_eff))
print("len ", len(nice_opt_eff))

In [ ]:
cleaned_event_container[nice_events[0]].muon.tel[1].efficiency.optical_efficiency

In [ ]:
print(nice_events)

In [ ]:
event

In [ ]:
source.subarray.index

In [ ]:
j

In [ ]:
np.mean(nice_opt_eff)

In [ ]:
###
# Calculate triggered telescopes
###
int_cut = 0.4
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
event_container = {}
t1 = time.perf_counter(), time.process_time()

tels_1_2 = []
tels_1_3 = []
tels_1_2_3 = []
tels = []

for k in range(78,100):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run39{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    for i,j in enumerate(event_iterator):
        checker += 1
        if len(j.trigger.tels_with_trigger) > 1:            
            tels.append(j.trigger.tels_with_trigger)
    t2 = time.perf_counter(), time.process_time()
        

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
# tels_1_2 = []
# tels_1_3 = []
# tels_1_4 = []
# tels_1_2_3 = []
# tels_1_2_4 = []
# tels_1_3_4 = []
# tels_1_2_3 = []
two = []
three = []
four = []
for i in tels:
    if len(i) == 2:
        two.append(i)
    elif len(i) == 3:
        three.append(i)
    else:
        four.append(i)

In [ ]:
len(two)

In [ ]:
fi_tw = 0
fi_th = 0
tw_th = 0
fi_fo = 0
tw_fo = 0
th_fo = 0
for i in two:
    if (i == [1,2]).all():
        fi_tw +=1
    elif (i == [2,3]).all():
        tw_th +=1
    elif (i == [1,3]).all():
        fi_th +=1
    elif (i == [1,4]).all():
        fi_fo +=1
    elif (i == [2,4]).all():
        tw_fo +=1
    elif (i == [3,4]).all():
        th_fo +=1

print("1-2" , fi_tw)
print("2-3" , tw_th)
print("1-3" , fi_th)
print("1-4" , fi_fo)
print("2-4" , tw_fo)
print("3-4" , th_fo)

In [ ]:
fi_tw_th = 0
fi_th_fo = 0
tw_th_fo = 0
fi_tw_fo = 0
for i in three:
    if (i == [1,2,3]).all():
        fi_tw_th +=1
    elif (i == [1,3,4]).all():
        fi_th_fo +=1
    elif (i == [2,3,4]).all():
        tw_th_fo +=1
    elif (i == [1,2,4]).all():
        fi_tw_fo +=1

print("1-2-3" , fi_tw_th)
print("1-3-4" , fi_th_fo)
print("2-3-4" , tw_th_fo)
print("1-2-4" , fi_tw_fo)

In [3]:
filename = f'/Users/vdk/CTA/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run3999.simtel.gz'
print(filename)
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)

Error loading ctapipe_io plugin: 'ctapipe_test_plugin:PluginEventSource'
Traceback (most recent call last):
  File "/Users/vdk/Software/ctasoft/ctapipe_test/ctapipe/src/ctapipe/core/plugins.py", line 21, in detect_and_import_plugins
    plugin = entry_point.load()
  File "/Users/vdk/mambaforge/envs/cta-test/lib/python3.9/site-packages/importlib_metadata/__init__.py", line 209, in load
    module = import_module(match.group('module'))
  File "/Users/vdk/mambaforge/envs/cta-test/lib/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 984, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'ctapipe_test_plugin'


/Users/vdk/CTA/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run3999.simtel.gz


In [4]:
event = next(event_iterator)

In [10]:
source.subarray.telescope_types

(TelescopeDescription(type='LST', optics_name='LST', camera_name='LSTCam'),
 TelescopeDescription(type='LST', optics_name='LST', camera_name='LSTCam'))

In [13]:
source.subarray.tels[]

TelescopeDescription(type='LST', optics_name='LST', camera_name='LSTCam')